In [ ]:
import os
from swarm import Swarm, Agent
from openai import OpenAI
from dotenv import load_dotenv

# Cargamos las variables de entorno
load_dotenv()

llm_client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY")
)
model = "llama3-groq-70b-8192-tool-use-preview"

def llm(prompt, model="llama3-groq-70b-8192-tool-use-preview"):
    chat_completion = llm_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=False,
        temperature=0,
        top_p=1,
        max_tokens=4096
    )
    return chat_completion.choices[0].message.content

print(llm("cuanto es 2+2?", model=model))

In [ ]:
# Función para chat
def chat(mensajes, model="llama3-groq-70b-8192-tool-use-preview"):
    """
    Realiza una conversación usando el modelo de chat de Groq.
    """
    chat_completion = llm_client.chat.completions.create(
        model=model,
        messages=mensajes
    )
    return chat_completion.choices[0].message.content

respuesta_chat = chat(mensajes = [
    {"role": "system", "content": "Eres un asistente útil, amigable y conciso"},
    {"role": "user", "content": "En breve ¿Cuáles son los beneficios de hacer ejercicio regularmente?"}
])
print("\nRespuesta del chat:")
print(respuesta_chat)

In [ ]:
from swarm import Swarm, Agent

agent_client = Swarm(client=llm_client)


In [ ]:
english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.",
    model=model,
    tool_choice="auto"
)
def transfer_to_english_agent():
    """Transfer english speaking users immediately."""
    return english_agent

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish.",
    model=model,
    tool_choice="auto"
)

def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent

agent = Agent(
    name="Welcome Agent",
    instructions="You have to choose the right agent based on the user's language.",
    model=model,
    tool_choice="auto",
    functions=[transfer_to_spanish_agent, transfer_to_english_agent]
)


messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = agent_client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])


In [ ]:
from swarm import Agent


def process_refund(item_id, reason="NOT SPECIFIED"):
    """Refund an item. Refund an item. Make sure you have the item_id of the form item_... Ask for user confirmation before processing the refund."""
    print(f"[mock] Refunding item {item_id} because {reason}...")
    return "Success!"


def apply_discount():
    """Apply a discount to the user's cart."""
    print("[mock] Applying discount...")
    return "Applied discount of 11%"


triage_agent = Agent(
    name="Triage Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
)
sales_agent = Agent(
    name="Sales Agent",
    instructions="Be super enthusiastic about selling bees.",
)
refunds_agent = Agent(
    name="Refunds Agent",
    instructions="Help the user with a refund. If the reason is that it was too expensive, offer the user a refund code. If they insist, then process the refund.",
    functions=[process_refund, apply_discount],
)


def transfer_back_to_triage():
    """Call this function if a user is asking about a topic that is not handled by the current agent."""
    return triage_agent


def transfer_to_sales():
    return sales_agent


def transfer_to_refunds():
    return refunds_agent


triage_agent.functions = [transfer_to_sales, transfer_to_refunds]
sales_agent.functions.append(transfer_back_to_triage)
refunds_agent.functions.append(transfer_back_to_triage)

In [ ]:
from swarm import Swarm, Agent

import os
from openai import OpenAI
from dotenv import load_dotenv

# Cargamos las variables de entorno
load_dotenv()

llm_client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY")
)
model = "llama3-groq-70b-8192-tool-use-preview"
client = Swarm(client=llm_client)

# Agente que busca en la base de datos
def search_database(query):
    # Simulación de búsqueda en la base de datos
    database = {
        "horarios": "Nuestros horarios son de 9am a 6pm.",
        "ubicación": "Estamos ubicados en la calle Principal 123."
    }
    return database.get(query.lower(), "No se encontró una respuesta en la base de datos.")

database_agent = Agent(
    name="Database Agent",
    instructions="Busca respuestas en la base de datos.",
    functions=[search_database]
)

# Agente de soporte al cliente
def customer_support(query):
    return "Derivando tu pregunta al soporte de cliente."

support_agent = Agent(
    name="Support Agent",
    instructions="Deriva preguntas al soporte de cliente.",
    functions=[customer_support]
)

# Agente de triage que decide a qué agente derivar
def triage_agent_function(messages):
    ...

triage_agent = Agent(
    name="Triage Agent",
    instructions="Decide a qué agente derivar la pregunta.",
    functions=[triage_agent_function]
)

# Función principal para ejecutar el agente
def run_agent(user_input):
    messages = [{"role": "user", "content": user_input}]
    response = client.run(agent=triage_agent, messages=messages)
    return response.messages[-1]["content"]

# -- streamlit --
import gradio as gr

def chat_interface(message):
    response = run_agent(message)
    return response

iface = gr.Interface(
    fn=chat_interface,
    inputs=gr.Textbox(lines=2, placeholder="Haz tu pregunta..."),
    outputs="text",
    title="Aplicación Multi-Agente con Swarm",
    description="Interactúa con nuestro sistema de agentes inteligentes."
)

iface.launch()